In [ ]:
%autosave 0

# ICPC 2016 UCSY

http://www.ucsy.edu.mm/ucsy/pages/2016_AR_Yangon_Contest_Problems.pdf

## Problem B (The Election)

There are several candidates and voters(citizon). Candidates issues manufasto, and voters choose the one best for him/her.

### Review of Modulo

$
(a + b) \bmod c => ((a \bmod c) + (b \bmod c)) \bmod c \\
(a * b) \bmod c => ((a \bmod c) * (b \bmod c)) \bmod c
$

This means that if **a** and **b** are big decimal integer and **c** is **10**, only lowest digits need to be calculated. It's also same if **a** and **b** are represented as *c-ary* number. If **c** is 16, and **a** and **b** are hexdecimal, lowest digit(0-F) need to be calculated.

In [ ]:
# decimal
a = 123
b = 456
print ((a+b) % 10, (3+6) % 10)
print ((a*b) % 10, (3*6) % 10)

# hexdecimal
a = 0x1234
b = 0x5678
print ((a+b) % 16, (4+8) % 16)
print ((a*b) % 16, (4*8) % 16)

# 1007-ary
BIG_N = 1007
a = BIG_N**2 * 1 + BIG_N * 2 + 3  # 123 (base-1007)
b = BIG_N**2 * 4 + BIG_N * 5 + 6  # 456 (base-1007)
print(a, b)  # for reference
print ((a+b) % BIG_N, (3+6) % BIG_N)
print ((a*b) % BIG_N, (3*6) % BIG_N)

### Moduler exponentiation

There is a good algorithm to calculate the modulo of big exponents ($x^n \bmod y$). Though this can not be used for this problem,because it requires not only modulo but also comparison.

In [ ]:
import math

def bits(integer):
    """ Gets number of bits in integer """
    result = 0
    while integer:
        integer >>= 1
        result += 1
    return result

def power(base, exponent, modulo=None):
    """ Allows fast exponentiation with and without modulo """
    result = 1
    if modulo == None:
        iteration = bits(exponent)
        while iteration >= 0:
            result *= result
            if (exponent >> iteration) & 1:
                result *= base
            iteration -= 1
    else:
        firstModulus = base % modulo
        iteration = bits(exponent)
        while iteration >= 0:
            result = (result * result) % modulo
            if (exponent >> iteration) & 1:
                result = (result * firstModulus) % modulo
            iteration -= 1

    return result


### Convert integer to the List of N-ary number

Following example returns the List of N-ary number (Little endian). (Take care not to overflow when converting other programming language)

In [ ]:
def int2n_ary(int_x, base_n):
    '''
    int_x: Number to be converted
    base_n: N of N-ary
    '''
    
    if int_x == 0:
        return [0]
    
    result = list()
    
    while int_x > 0:
        modulo = int_x % base_n
        result.append(modulo)
        int_x //= base_n
        
    return result

In [ ]:
def test_int2n_ary():
    print(int2n_ary(0x1234abcde, 16))
    print(int2n_ary(100, 3))
    print(int2n_ary(1027, 2))
    print(int2n_ary(1024, 2))
    print(int2n_ary(100, 100))
    print(int2n_ary(3**100, 1007))
    
test_int2n_ary()

### Product - Result of Multiply (**P**) of k-digits number(**N**) and 1-digit number(**M**)

$N_{k-1} N_{k-2} ... N_2 N_1 N_0 \cdot M = P_k P_{k-1} P_{k-2} ... P_2 P_1 P_0$

For example: $12345678 \cdot 9 = 111111102\\
P_0 = (8 \cdot 9) \bmod 10\\
P_1 = (7 \cdot 9 + (8 \cdot 9 \div 10)) \bmod 10$

In [ ]:
def production_of_N_array_and_1_digit(array_of_int2n_ary, base_n, int_less_than_n):
    '''
    array_of_int2n_ary: output of int2n_ary
    base_n: same input to int2n_ary
    int_less_than_n: Integer less than base_n
    
    Output: None
    Multiply array_of_int2n_ary and int_less_than_n then update array_of_int2n_ary
    
    This function is same as prod_bigN() in prob_b1.py
    '''
    overflow = 0
    for idx in range(len(array_of_int2n_ary)):
        temp = array_of_int2n_ary[idx] * int_less_than_n + overflow
        overflow = temp // base_n
        array_of_int2n_ary[idx] = temp % base_n

    if overflow != 0:
        array_of_int2n_ary.append(overflow)

#    print(array_of_int2n_ary)

    return


In [ ]:
def test_prod():
    BIG_N = 10**9+7
    
    n_array = int2n_ary(1, BIG_N)  # initialize to 1
    # calculate 12345678 ** 10, result is in BIG_N-ary array
    for i in range(10):
        production_of_N_array_and_1_digit(n_array, BIG_N, 12345678)
        print(n_array)
        result = 12345678 ** (i+1)
        print (result % BIG_N, (result // BIG_N) % BIG_N, (result // BIG_N**2) % BIG_N, (result // BIG_N**3) % BIG_N)
        
test_prod()

### prob_b1.py

Modifyied to open input file explicitly, so that IPython can handle it.

In [ ]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-

'''
    2016 ICPC at UCSY
    Problem-B: The Election (without unlimited interger version)
'''
import sys
BIG_PRIME_NUMBER = 10**9+7


class TestCase():
    pass


def parse_tc(tc):
    '''
        Input: class of Test Case
        Update: n, x_i, q, query in test case
        Return: None
    '''

    tc.query = list()

    tc.n = int(tc.infile.readline())
    tc.x_i = list(map(int,tc.infile.readline().split()))
    if (len(tc.x_i) != tc.n):
        print('Invalid input: number of account')
        assert False
    tc.q = int(tc.infile.readline())
    for i in range(tc.q):
        s = tc.infile.readline()
        query = list(map(int,s.split()))

        if (query[0] == 1):
            if len(query) != 5:
                print('Invalid query-1 data', s)
                assert False
        elif (query[0] == 2):
            if len(query) != 2:
                print('Invalid query-2 data', s)
                assert False
        else:
            print('Unknown query data', s)
            assert False

        tc.query.append(query)

    return


def prod_bigN(gain, rate):

    overflow = 0
    for idx in range(len(gain)):
        temp = gain[idx] * rate + overflow
        overflow = temp // BIG_PRIME_NUMBER
        gain[idx] = temp % BIG_PRIME_NUMBER

    if overflow != 0:
        gain.append(overflow)

#    print(gain)

    return


def calc_gain(base, rate, exp):

    gain = [base]

    for i in range(exp):
        prod_bigN(gain, rate)

    return gain


def is_bigger(a, b):
    '''
    a: list of n-ary number
    b: list of n-ary number
    return True if a>b
           False otherwise
    '''

    if len(a) > len(b):
        return True
    elif len(b) > len(a):
        return False
    else:               # same length
        idx = len(a) - 1
        while idx >= 0:
            if a[idx] > b[idx]:
                return True
            elif b[idx] > a[idx]:
                return False
            idx -= 1

        return False    # same number

    assert False        # never happen

def search_best(tc, voter, manufesto):
    best_gain = []
    best_candidate = 0

    for m in manufesto:
        if voter > m[1] and voter <= m[2]:
            gain = calc_gain(tc.x_i[voter-1], m[3], voter - m[1])
            if is_bigger(gain, best_gain):
                best_gain = gain
                best_candidate = m[4]

    if best_gain == []:
        print(-1)
    else:
        print(best_candidate, best_gain[0])

    return

def solve(tc_num, tc):
    '''
        Input: Test Case
        Return: None
    '''

    print('Case ',tc_num+1,':',sep='')
    parse_tc(tc)
    manufesto = list()
    for q in tc.query:
        if q[0] == 1:  ## Manufesto
            manufesto.append(q)
        elif q[0] == 2:  ## voter
            search_best(tc, q[1], manufesto)
        else:
            print('Logic error')
            assert False

    return


##
##  Main routine
##
if __name__ == '__main__':
    tc = TestCase()
#    tc.infile = sys.stdin
    tc.infile = open('b.in', 'r') 
    tc.t = int(tc.infile.readline())
    
    for i in range(tc.t):
        solve(i, tc)

    if tc.infile != sys.stdin:
        tc.infile.close()